# YOUR PROJECT TITLE

For this assignment, I will be using two data sets that are relevant to agriculture and weather in the United States, specifically North Dakota, being one of the biggest wheat producing states in the US. The first data set contains information on yearly corn yield per acre, from around 1900 to 2022. This data is from USDA National Agricultural Statistics Service (NASS). The second data set contains yearly average temperature, precipitation, and the date of the first freeze, from a central weather station in North Dakota, with data ranging from a similar interval as the agricultural data. This data is sourced from the National Oceanic and Atmospheric Administration (NOAA). By combining these data sets, the hope is to investigate the relationship between weather patterns and corn production in the North Dakota over the past century.

Imports and set magics:

In [314]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

To begin, let's read the first data set on wheat yield and inspect it:

In [315]:
NDyield = pd.read_csv('ND_yield.csv')

print(NDyield.head(10))


  Program  Year               Period  Week Ending Geo Level         State  \
0  SURVEY  2022                 YEAR          NaN     STATE  NORTH DAKOTA   
1  SURVEY  2021                 YEAR          NaN     STATE  NORTH DAKOTA   
2  SURVEY  2020                 YEAR          NaN     STATE  NORTH DAKOTA   
3  SURVEY  2019                 YEAR          NaN     STATE  NORTH DAKOTA   
4  SURVEY  2018                 YEAR          NaN     STATE  NORTH DAKOTA   
5  SURVEY  2017                 YEAR          NaN     STATE  NORTH DAKOTA   
6  SURVEY  2016                 YEAR          NaN     STATE  NORTH DAKOTA   
7  SURVEY  2015                 YEAR          NaN     STATE  NORTH DAKOTA   
8  SURVEY  2015  YEAR - AUG FORECAST          NaN     STATE  NORTH DAKOTA   
9  SURVEY  2015  YEAR - JUL FORECAST          NaN     STATE  NORTH DAKOTA   

   State ANSI  Ag District  Ag District Code  County  ...  Zip Code  Region  \
0          38          NaN               NaN     NaN  ...       NaN     N

As can be seen, we have a lot of unnecessary information, that should be dropped for simplicity and to make the data set easier to manage. More importantly, the data also contains forecasts, and these are not needed, so should be deleted first: 


In [316]:
I = NDyield.Period.str.contains('FORECAST')
NDyield = NDyield.loc[I == False] # keep everything else

Next, let's make the data set narrower by dropping everything but the year and the average yield per acre (Most other variables have the same value in each row anyways, since all the data is from North Dakota)

In [317]:
drop_variables_yield = ['Program', 'Period', 'Week Ending','Geo Level','State','State ANSI','Ag District','Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed','Commodity','Data Item', 'Domain', 'Domain Category', 'CV (%)']

NDyield = NDyield.drop(columns = drop_variables_yield)

print(NDyield.head(10))

    Year  Value
0   2022   48.9
1   2021   32.2
2   2020   47.6
3   2019   48.4
4   2018   47.6
5   2017   37.9
6   2016   45.0
7   2015   46.7
10  2014   46.3
11  2013   45.4


Now we are left with the information relevant to this project, let's continue by importing the weather data:

In [318]:
NDweather = pd.read_csv('ND_weather.csv')

print(NDweather.head(5))

       STATION                   NAME  DATE  FZF0  PRCP  TAVG
0  USC00326365  NEW SALEM 5 NW, ND US  1923  30.0   NaN  40.9
1  USC00326365  NEW SALEM 5 NW, ND US  1924   NaN   NaN   NaN
2  USC00326365  NEW SALEM 5 NW, ND US  1925  27.0   NaN  43.0
3  USC00326365  NEW SALEM 5 NW, ND US  1926  30.0   NaN   NaN
4  USC00326365  NEW SALEM 5 NW, ND US  1927   NaN   NaN   NaN


Again we have some variables that can be dropped, namely 'NAME' and 'STATION', that are the same for each row.

In [319]:
drop_variables_weather = ['STATION', 'NAME']
NDweather = NDweather.drop(columns=drop_variables_weather)
print(NDweather.head(10))

   DATE  FZF0   PRCP  TAVG
0  1923  30.0    NaN  40.9
1  1924   NaN    NaN   NaN
2  1925  27.0    NaN  43.0
3  1926  30.0    NaN   NaN
4  1927   NaN    NaN   NaN
5  1928  30.0    NaN  43.2
6  1929   NaN    NaN   NaN
7  1931  28.0    NaN   NaN
8  1932  32.0    NaN  41.4
9  1933  30.0  12.86  42.5


There also seems to be a lot of missing values. To check for these, the .isna function can help:

In [320]:
NDweather[NDweather.isna().any(axis=1)]


,DATE,FZF0,PRCP,TAVG
0,1923,30.0,NaN,40.9
1,1924,NaN,NaN,NaN
2,1925,27.0,NaN,43.0
3,1926,30.0,NaN,NaN
4,1927,NaN,NaN,NaN
5,1928,30.0,NaN,43.2
6,1929,NaN,NaN,NaN
7,1931,28.0,NaN,NaN
8,1932,32.0,NaN,41.4
13,1937,32.0,NaN,38.9


In [321]:
NDweather = NDweather.iloc[98:32:-1].reset_index(drop=True)
print(NDweather.head(100))

    DATE  FZF0   PRCP  TAVG
0   2022  16.0  21.21  40.0
1   2021  30.0  14.49  44.3
2   2020  27.0  10.66  42.7
3   2019  31.0  27.69  38.0
4   2018  29.0  17.93  39.9
..   ...   ...    ...   ...
61  1961  30.0  13.44  42.3
62  1960  30.0  14.92  41.8
63  1959  22.0  10.95  41.0
64  1958  32.0  10.57  42.4
65  1957  30.0  19.73  41.6

[66 rows x 4 columns]


In [322]:
NDweather[NDweather.isna().any(axis=1)]

,DATE,FZF0,PRCP,TAVG


In [323]:
NDweather = NDweather.drop(index=range(1), axis=0)
print(NDweather.head(100))

    DATE  FZF0   PRCP  TAVG
1   2021  30.0  14.49  44.3
2   2020  27.0  10.66  42.7
3   2019  31.0  27.69  38.0
4   2018  29.0  17.93  39.9
5   2017  30.0  12.50  41.7
..   ...   ...    ...   ...
61  1961  30.0  13.44  42.3
62  1960  30.0  14.92  41.8
63  1959  22.0  10.95  41.0
64  1958  32.0  10.57  42.4
65  1957  30.0  19.73  41.6

[65 rows x 4 columns]


We also have some rows with NaN values. To deal with these let's drop the first 35 rows (since they contain a lot of NaN's). Note that since the data sets starts the same year, the same number of rows should be deleted from each of them:

In [324]:
NDweather = NDweather.drop(index=range(35), axis=0)
NDyield   = NDyield.drop(index=range(35), axis=0)
NDweather.reset_index(inplace = True, drop = True)
NDyield.reset_index(inplace = True, drop = True)
print(NDweather.head(100))


KeyError: '[0] not found in axis'

Finally the last row, 2023, also has NaN values, so it is dropped too:

In [ ]:
print(NDyield.head(100))

    Year  Value
0   1997   23.8
1   1997   24.5
2   1996   31.6
3   1996   29.8
4   1996   30.0
..   ...    ...
95  1971   31.8
96  1971   30.8
97  1971   29.3
98  1971   31.5
99  1970   23.6

[100 rows x 2 columns]


In [ ]:
NDweather.drop(index=NDweather.index[-1],axis=0,inplace=True)
NDyield.drop(index=NDyield.index[-1],axis=0,inplace=True)

AttributeError: 'NoneType' object has no attribute 'drop'

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

NameError: name 'venn2' is not defined

<Figure size 1500x700 with 0 Axes>

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.